In [39]:
import numpy as np
from scipy import stats as st
from matplotlib import pyplot as plt

%matplotlib inline

In [40]:
n = 100
teta = 2

Генерация выборки

In [41]:
data = np.power(np.random.rand(n), 1 / (1 - teta))
data

array([ 2.94326539,  1.00834124,  9.67166598,  4.99144181,  1.32430622,
        2.44823894,  2.71173605,  3.55755048,  1.3071978 ,  4.29150979,
        1.65588817,  6.93199257,  1.7304012 ,  1.26855646,  4.81519764,
        9.80819533,  1.80356272,  1.57917453,  3.10464579,  4.55277119,
        3.38309083,  1.8043798 ,  1.20173656,  1.03243104, 12.40255409,
        2.8190704 ,  6.92184469, 24.71015462,  2.08022266,  1.99675401,
        1.1240385 ,  1.99857564,  3.84011853,  2.56964713,  1.18229325,
        1.36246639,  1.22077079,  2.81143196,  1.07656767,  2.04756906,
        1.09274034,  1.09621394,  1.53948146,  6.0076231 ,  4.75119457,
        1.5503766 ,  1.32508971,  1.3615539 ,  1.92298291,  1.39681947,
        1.20203698,  3.50410905,  2.13965528,  1.71372043,  1.1547032 ,
        1.14114004,  1.17568665,  1.08661658,  6.43585405,  1.21766304,
        1.56871523,  3.908006  ,  9.94047208,  1.08402532,  2.6253209 ,
        1.02188967,  1.65189096,  2.35372928, 25.53717881,  4.76

вычисление полученных аналитически доверительных интервалов

In [42]:
beta = 0.95
t1 = -1.96
t2 = 1.96

оценка

In [43]:
assessment_teta = (n / np.sum(np.log(data))) + 1
assessment_teta

2.052298473581029

In [44]:
h1 = assessment_teta + t1 * (assessment_teta - 1) / n**0.5
h2 = assessment_teta + t2 * (assessment_teta - 1) / n**0.5
print(f"h1 - {h1}\nh2 - {h2}\nдлина интервала - {h2 - h1}")
an_res_teta = (h1,h2)

h1 - 1.8460479727591472
h2 - 2.2585489744029106
длина интервала - 0.41250100164376335


оценка медианы

In [45]:
assessment_med = np.power(2, 1 / (assessment_teta - 1))
assessment_med

1.9322753892712774

In [46]:
h1 = t1 * assessment_med * np.log(2) / (assessment_teta - 1) / np.sqrt(n) + assessment_med
h2 = t2 * assessment_med * np.log(2) / (assessment_teta - 1) / np.sqrt(n) + assessment_med
print(f"h1 - {h1}\nh2 - {h2}\nдлина интервала - {h2 - h1}")
an_res_med = (h1,h2)

h1 - 1.6828092451440786
h2 - 2.181741533398476
длина интервала - 0.4989322882543976


Непараметрический бутстрап для оценки теты

In [47]:
func = lambda x: (n / np.sum(np.log(x))) + 1 - assessment_teta
non_parametric_bootstrap_teta = st.bootstrap(
    (data, ), 
    func, 
    n_resamples=1_000, 
    method='basic',
    confidence_level=0.95
).confidence_interval

non_parametric_bootstrap_teta = (assessment_teta - non_parametric_bootstrap_teta.high, assessment_teta - non_parametric_bootstrap_teta.low)
print("доверительный интервал - ",non_parametric_bootstrap_teta, "\nдлина интервала -",non_parametric_bootstrap_teta[1] - non_parametric_bootstrap_teta[0])

доверительный интервал -  (1.8939236902890424, 2.2494639209754705) 
длина интервала - 0.35554023068642815


Параметрический бутстрап для оценки теты

In [48]:
from collections.abc import Callable
from typing import NamedTuple


class ConfidenceInterval(NamedTuple):
    low: float
    high: float


def parametric_bootstrap(
    data: np.ndarray,
    statistic: Callable[[np.ndarray], float],
    prob_model: Callable[[int], np.ndarray],
    n_resamples: int = 10_000,
    confidence_level: float = 0.95,
) -> ConfidenceInterval:
    n = len(data)
    bootstrap_data = np.ndarray((n_resamples, ), dtype=np.float32)

    for i in range(n_resamples):
        resample = prob_model(n)
        bootstrap_data[i] = statistic(resample)
    
    bootstrap_data.sort()

    return ConfidenceInterval(
        low=bootstrap_data[round((1 - confidence_level) / 2 * n_resamples) - 1],
        high=bootstrap_data[round((1 + confidence_level) / 2 * n_resamples) - 1]
    )


parametric_bootstrap_teta = parametric_bootstrap(
    data,
    func,
    lambda n: np.power(np.random.rand(n), 1 / (1 - assessment_teta)),
    n_resamples=50_000, 
    confidence_level=0.95
)

parametric_bootstrap_teta = (assessment_teta - parametric_bootstrap_teta.high, assessment_teta - parametric_bootstrap_teta.low)
print("доверительный интервал - ",parametric_bootstrap_teta, "\nдлина интервала -",parametric_bootstrap_teta[1] - parametric_bootstrap_teta[0])

доверительный интервал -  (1.812529089316083, 2.2287761293494235) 
длина интервала - 0.41624704003334045


Непараметрический бутстрап для оценки медианы

In [49]:
func = lambda x: np.power(2, 1 / (n / np.sum(np.log(x)))) - assessment_med
non_parametric_bootstrap_med = st.bootstrap(
    (data, ), 
    func, 
    n_resamples=1_000, 
    method='basic',
    confidence_level=0.95
).confidence_interval

non_parametric_bootstrap_med  = (assessment_med - non_parametric_bootstrap_med .high, assessment_med - non_parametric_bootstrap_med .low)
print("доверительный интервал - ",non_parametric_bootstrap_med, "\nдлина интервала -",non_parametric_bootstrap_med[1] - non_parametric_bootstrap_med[0])

доверительный интервал -  (1.7259015902378363, 2.1798902641526405) 
длина интервала - 0.45398867391480424


Параметрический бутстрап для оценки медианы

In [50]:
from collections.abc import Callable
from typing import NamedTuple


class ConfidenceInterval(NamedTuple):
    low: float
    high: float


def parametric_bootstrap(
    data: np.ndarray,
    statistic: Callable[[np.ndarray], float],
    prob_model: Callable[[int], np.ndarray],
    n_resamples: int = 10_000,
    confidence_level: float = 0.95,
) -> ConfidenceInterval:
    n = len(data)
    bootstrap_data = np.ndarray((n_resamples, ), dtype=np.float32)

    for i in range(n_resamples):
        resample = prob_model(n)
        bootstrap_data[i] = statistic(resample)
    
    bootstrap_data.sort()

    return ConfidenceInterval(
        low=bootstrap_data[round((1 - confidence_level) / 2 * n_resamples) - 1],
        high=bootstrap_data[round((1 + confidence_level) / 2 * n_resamples) - 1]
    )


parametric_bootstrap_med = parametric_bootstrap(
    data,
    func,
    lambda n: np.power(np.random.rand(n), 1 / (1 - assessment_teta)),
    n_resamples=50_000, 
    confidence_level=0.95
)

parametric_bootstrap_med = (assessment_teta - parametric_bootstrap_med.high, assessment_teta - parametric_bootstrap_med.low)
print("доверительный интервал - ",parametric_bootstrap_med, "\nдлина интервала -",parametric_bootstrap_med[1] - parametric_bootstrap_med[0])

доверительный интервал -  (1.7770338229977618, 2.275873707994176) 
длина интервала - 0.4988398849964142


Сравнение

In [51]:
print(
    "Для теты",
    f"{an_res_teta = }",
    f"{non_parametric_bootstrap_teta = }",
    f"{parametric_bootstrap_teta = }",
    "\nДля медианы",
    f"{an_res_med = }",
    f"{non_parametric_bootstrap_med = }",
    f"{parametric_bootstrap_med = }",
    sep="\n\n"
)

Для теты

an_res_teta = (1.8460479727591472, 2.2585489744029106)

non_parametric_bootstrap_teta = (1.8939236902890424, 2.2494639209754705)

parametric_bootstrap_teta = (1.812529089316083, 2.2287761293494235)


Для медианы

an_res_med = (1.6828092451440786, 2.181741533398476)

non_parametric_bootstrap_med = (1.7259015902378363, 2.1798902641526405)

parametric_bootstrap_med = (1.7770338229977618, 2.275873707994176)
